<a href="https://colab.research.google.com/github/sumitk87549/AIAYN/blob/main/AIAYN_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-3
eval_iters = 250
n_embed = 384
n_head = 4
n_layer = 4
dropout = 0.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open("/content/drive/MyDrive/Assets/wizard_of_oz.txt", 'r', encoding='utf-8') as f:
  text = f.read()
  chars = sorted(list(set(text)))

vocab_size = len(chars)

In [ ]:
string_to_int = { ch:i for i,ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: [int_to_string[i] for i in l]

data = torch.tensor(encode(text), dtype=torch.long)



In [ ]:
data[:10]

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49])

In [ ]:
n = int(0.8*len(data))
# The first 80% of the data becomes the training set.
train_data = data[:n]
# The remaining 20% becomes the validation set.
val_data = data[n:]

def get_batch(split):
  data = train_data if split=='train' else val_data
  #  generates batch_size random indices (ix) within the valid range of the data. The upper bound is len(data) - block_size
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  # x,y = x.to(device), y.to(device)
  return x,y



In [ ]:
@torch.no_grad()
def estimate_loss():
  out={}
  model.eval()
  for split in ["train", "val"]:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X,Y= get_batch(split)
      logits, loss = model(X,Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
    model.train()
  return out

In [ ]:
class GPTLanguageModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    # Tokenized Inputs
    self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
    # Embeddings + Pos Encoding
    self.position_embedding_table = nn.Embedding(block_size, n_embed)
    # Sequential NN with 4(n_layer) Decoder Layers
    self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
    # Normalization Layer
    self.ln_f = nn.LayerNorm(n_embed)

    self.lm_head = nn.Linear(n_embed, vocab_size)


  def forward(self, index, targets=None):
    logits = self.token_embedding_table(index)

    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, index, max_new_tokens):
    # index is (B,T) array of indices in the current context
    for _ in range(max_new_tokens):
      # Generate predictions
      logits, loss = self.forward(index)
      # focus only on the last time step
      logits = logits[:, -1, :] # becomes (B,C)
      # apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B,C)
      # sample from distribution
      index_next = torch.multinomial(probs, num_samples=1) # (B,1)
      # append sampled index to the running sequence
      index = torch.cat((index, index_next), dim=1) # (B,T+1)
    return index

model = GPTLanguageModel(vocab_size)
# m = model.to(device)






In [ ]:
# AdamW Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
  if iter % eval_iters == 0:
    losses = estimate_loss()
    print(f"step: {iter}, train_loss: {losses['train']:.6f}, validation_loss:{losses['val']:.6f}")
    # print(losses)
  # sample a batch of data
  xb, yb = get_batch('train')

  # evaluate the loss
  logits,loss = model.forward(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())



step: 0, train_loss: 3.227675, validation_loss:3.255366
step: 250, train_loss: 3.018565, validation_loss:3.038089
step: 500, train_loss: 2.847684, validation_loss:2.886240
step: 750, train_loss: 2.754371, validation_loss:2.790671
2.7137367725372314
